In [24]:
#Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sys
import json
import matplotlib.pyplot as plt
import os
from google.colab import drive

In [25]:
# --- Load and Preprocess Data ---
def load_and_preprocess_data(file_path):
    """Loads and preprocesses the Aave V2 transaction data with robust schema handling."""
    print("\nLoading data...")
    if not os.path.exists(file_path):
        print(f"Error: Input file not found at the specified path: {file_path}", file=sys.stderr)
        return None

    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {file_path}. Please check the file format.", file=sys.stderr)
        return None

    df = pd.json_normalize(data, sep='_')
    print("Preprocessing data...")

    # Rename fields for consistency
    df.rename(columns={
        'userWallet': 'wallet',
        'action': 'type',
        'actionData_amount': 'amount',
        'actionData_assetSymbol': 'asset_symbol',
        'actionData_assetPriceUSD': 'asset_price_usd'
    }, inplace=True)

    # Type conversions
    df['timestamp']       = pd.to_datetime(df['timestamp'], unit='s')
    df['amount']          = pd.to_numeric(df['amount'], errors='coerce')
    df['asset_price_usd'] = pd.to_numeric(df['asset_price_usd'], errors='coerce')

    # Drop malformed rows
    df.dropna(subset=['wallet','amount','asset_price_usd','asset_symbol'], inplace=True)

    # Compute USD amount based on token decimals
    df['decimals']     = df['asset_symbol'].apply(get_token_decimals)
    df['amount_usd']   = (df['amount'] / (10**df['decimals'])) * df['asset_price_usd']

    # Handle liquidation calls
    liquid = df['type'] == 'liquidationcall'
    if liquid.any():
        # Determine debt symbol column
        if 'actionData_debtAssetSymbol' in df.columns:
            debt_sym_col = 'actionData_debtAssetSymbol'
        elif 'actionData_debtToken' in df.columns:
            debt_sym_col = 'actionData_debtToken'
        else:
            debt_sym_col = None

        # Determine debt amount column
        debt_amt_col = 'actionData_debtToCover' if 'actionData_debtToCover' in df.columns else None

        if debt_sym_col and debt_amt_col:
            # Recompute decimals for debt asset
            df.loc[liquid, 'decimals'] = (
                df.loc[liquid, debt_sym_col]
                  .apply(get_token_decimals)
            )
            # Recompute USD amount based on debt to cover
            df.loc[liquid, 'amount_usd'] = (
                pd.to_numeric(df.loc[liquid, debt_amt_col], errors='coerce')
                / (10 ** df.loc[liquid, 'decimals'])
            ) * df.loc[liquid, 'asset_price_usd']
        else:
            print(
                "⚠️  Warning: JSON schema for liquidation calls is missing expected fields. "
                "Skipping debt‐side USD re‐calculation.",
                file=sys.stderr
            )

    print(f"Loaded and processed {len(df)} transactions for {df['wallet'].nunique()} unique wallets.")
    return df

In [26]:
# --- Feature Engineering ---
def engineer_features(df):
    """Engineers features for each wallet."""
    print("Engineering features for each wallet...")
    features = df.groupby('wallet').agg(
        first_tx=('timestamp', 'min'),
        last_tx=('timestamp', 'max'),
        transaction_count=('txHash', 'nunique'),
    )

    # Monetary summaries
    features['total_deposited_usd']    = df[df['type']=='deposit'].groupby('wallet')['amount_usd'].sum()
    features['total_borrowed_usd']     = df[df['type']=='borrow'].groupby('wallet')['amount_usd'].sum()
    features['total_repaid_usd']       = df[df['type']=='repay'].groupby('wallet')['amount_usd'].sum()
    features['total_redeemed_usd']     = df[df['type']=='redeemunderlying'].groupby('wallet')['amount_usd'].sum()

    # Action counts
    action_counts = df.groupby(['wallet','type']).size().unstack(fill_value=0)
    features = features.join(action_counts, how='left').fillna(0)

    # Age and activity
    features['wallet_age_days'] = (features['last_tx'] - features['first_tx']).dt.days
    features.loc[features['wallet_age_days']==0, 'wallet_age_days'] = 1
    features['tx_per_day'] = features['transaction_count'] / features['wallet_age_days']

    # Ratios
    eps = 1e-6
    features['borrow_to_deposit_ratio'] = (
        features['total_borrowed_usd'] / (features['total_deposited_usd'] + eps)
    )
    features['repayment_to_borrow_ratio'] = (
        features.get('repay',0) / (features.get('borrow',0) + eps)
    )

    # Liquidation count
    if 'liquidationcall' in features.columns:
        features.rename(columns={'liquidationcall':'liquidation_count'}, inplace=True)
    else:
        features['liquidation_count'] = 0

    print("Feature engineering complete.")
    return features.reset_index()


In [27]:
# --- Scoring Model ---
def generate_scores(features_df):
    """Generates a credit score from 0 to 1000."""
    print("Generating credit scores...")
    # Sub‐scores
    health    = 1 - np.log1p(features_df['borrow_to_deposit_ratio'])
    stability = np.log1p(features_df['wallet_age_days']) + features_df['repayment_to_borrow_ratio']
    activity  = 1 / (1 + np.abs(features_df['tx_per_day'] - 1))

    # Weighted raw score
    raw_score = (0.4 * health) + (0.4 * stability) + (0.2 * activity)

    # Scale to [100,1000]
    scaler = MinMaxScaler(feature_range=(100,1000))
    scaled = scaler.fit_transform(raw_score.values.reshape(-1,1)).flatten()

    # Zero out liquidations
    features_df['credit_score'] = scaled
    features_df.loc[features_df['liquidation_count']>0, 'credit_score'] = 0
    features_df['credit_score'] = features_df['credit_score'].astype(int)

    print("Scoring complete.")
    return features_df[['wallet','credit_score']]


In [28]:
# --- Analysis & Visualization ---
def create_analysis_report(scores_df, output_dir):
    """Generates and saves a score distribution histogram."""
    print("Generating analysis report...")
    plt.figure(figsize=(12,7))
    plt.hist(scores_df['credit_score'], bins=20)
    plt.title('Distribution of Wallet Credit Scores')
    plt.xlabel('Credit Score (0-1000)')
    plt.ylabel('Number of Wallets')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    os.makedirs(output_dir, exist_ok=True)
    plot_path = os.path.join(output_dir, 'score_distribution.png')
    plt.savefig(plot_path)
    plt.close()
    print(f"Score distribution plot saved to {plot_path}")

In [30]:
print("All column names in the DataFrame:")
print(df.columns.tolist())

print("Unique action types:")
print(df['type'].unique())


All column names in the DataFrame:
['wallet', 'network', 'protocol', 'txHash', 'logId', 'timestamp', 'blockNumber', 'type', '__v', '_id_$oid', 'actionData_type', 'amount', 'asset_symbol', 'asset_price_usd', 'actionData_poolId', 'actionData_userId', 'createdAt_$date', 'updatedAt_$date', 'actionData_toId', 'actionData_borrowRateMode', 'actionData_borrowRate', 'actionData_variableTokenDebt', 'actionData_stableTokenDebt', 'actionData_callerId', 'actionData_useATokens', 'actionData_repayerId', 'actionData_liquidatorId', 'actionData_collateralAmount', 'actionData_collateralAssetPriceUSD', 'actionData_principalAmount', 'actionData_borrowAssetPriceUSD', 'actionData_collateralReserveId', 'actionData_collateralReserveSymbol', 'actionData_principalReserveId', 'actionData_principalReserveSymbol', 'decimals', 'amount_usd']
Unique action types:
['deposit' 'redeemunderlying' 'borrow' 'repay' 'liquidationcall']


In [29]:

if __name__ == "__main__":
    # Path to your JSON file and output generated in Google Drive
    file_path = "/content/drive/MyDrive/Tranction_Wallet/user-wallet-transactions.json"
    output_dir = "/content/drive/MyDrive/Tranction_Wallet"

    # 1. Load & preprocess
    df = load_and_preprocess_data(file_path)
    if df is None:
        sys.exit(1)

    # 2. Feature engineering
    feats = engineer_features(df)

    # 3. Scoring
    scores = generate_scores(feats)

    # 4. Save results
    os.makedirs(output_dir, exist_ok=True)
    scores.sort_values('credit_score', ascending=False)
    scores.to_csv(os.path.join(output_dir, 'wallet_scores.csv'), index=False)
    print(f"\nSuccess! Wallet scores saved to {output_dir}/wallet_scores.csv")

    # 5. Analysis report
    create_analysis_report(scores, output_dir)
    print("\nDone. Check the 'output' folder for results.")



Loading data...
Preprocessing data...


⚠️  Warning: JSON schema for liquidation calls is missing expected fields. Skipping debt‐side USD re‐calculation.


Loaded and processed 100000 transactions for 3497 unique wallets.
Engineering features for each wallet...
Feature engineering complete.
Generating credit scores...
Scoring complete.

Success! Wallet scores saved to /content/drive/MyDrive/Tranction_Wallet/wallet_scores.csv
Generating analysis report...
Score distribution plot saved to /content/drive/MyDrive/Tranction_Wallet/score_distribution.png

Done. Check the 'output' folder for results.
